# Spark DataFrames Project Exercise 

Let's get some quick practice with your new Spark DataFrame skills, you will be asked some basic questions about some stock market data, in this case Walmart Stock from the years 2012-2017. This exercise will just ask a bunch of questions, unlike the future machine learning exercises, which will be a little looser and be in the form of "Consulting Projects", but more on that later!

For now, just answer the questions and complete the tasks below.

#### Use the walmart_stock.csv file to Answer and complete the  tasks below!

#### Start a simple Spark Session

In [70]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [71]:
df = spark.read.csv("/HdiNotebooks/walmart_stock.csv", header=True, inferSchema=True)

#### What are the column names?

In [72]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### What does the Schema look like?

In [73]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)

#### Print out the first 5 columns.

In [74]:
df.show(5)

+--------+---------+---------+---------+---------+--------+---------+
|    Date|     Open|     High|      Low|    Close|  Volume|Adj Close|
+--------+---------+---------+---------+---------+--------+---------+
|1/3/2012|59.970001|61.060001|59.869999|60.330002|12668800|52.619235|
|1/4/2012|60.209999|60.349998|59.470001|59.709999| 9593300|52.078475|
|1/5/2012|59.349998|59.619999|58.369999|59.419998|12768200|51.825539|
|1/6/2012|59.419998|59.450001|58.869999|     59.0| 8069400| 51.45922|
|1/9/2012|59.029999|59.549999|58.919998|    59.18| 6679300|51.616215|
+--------+---------+---------+---------+---------+--------+---------+
only showing top 5 rows

#### Date field does not infer to temporal type
#### Redo with explicit schema and date formatting

In [75]:
from pyspark.sql.types import StructType, StructField, DateType, FloatType, IntegerType

schema = StructType([StructField('Date', DateType(), nullable=True),
                     StructField('Open', FloatType(), nullable = True),
                     StructField('High', FloatType(), nullable = True),
                     StructField('Low', FloatType(), nullable=True),
                     StructField('Close', FloatType(), nullable = True),
                     StructField('Volume', IntegerType(), nullable = True),
                     StructField('Adj Close', FloatType(), nullable = True)])

df = spark.read.options(header="true").option("dateFormat", "MM/dd/YYYY").schema(schema).csv("/HdiNotebooks/walmart_stock.csv")

In [76]:
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: float (nullable = true)
 |-- High: float (nullable = true)
 |-- Low: float (nullable = true)
 |-- Close: float (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: float (nullable = true)

In [77]:
# Note corrected date
df.show(10)

+----------+-----+-----+-----+-----+--------+---------+
|      Date| Open| High|  Low|Close|  Volume|Adj Close|
+----------+-----+-----+-----+-----+--------+---------+
|2012-01-03|59.97|61.06|59.87|60.33|12668800|52.619236|
|2012-01-04|60.21|60.35|59.47|59.71| 9593300|52.078476|
|2012-01-05|59.35|59.62|58.37|59.42|12768200| 51.82554|
|2012-01-06|59.42|59.45|58.87| 59.0| 8069400| 51.45922|
|2012-01-09|59.03|59.55|58.92|59.18| 6679300|51.616215|
|2012-01-10|59.43|59.71|58.98|59.04| 6907300| 51.49411|
|2012-01-11|59.06|59.53|59.04| 59.4| 6365600|51.808098|
|2012-01-12|59.79| 60.0| 59.4| 59.5| 7236400|51.895317|
|2012-01-13|59.18|59.61|59.01|59.54| 7729300|51.930202|
|2012-01-17|59.87|60.11|59.52|59.85| 8500000| 52.20058|
+----------+-----+-----+-----+-----+--------+---------+
only showing top 10 rows

#### Use describe() to learn about the DataFrame.

In [78]:
df.describe()


DataFrame[summary: string, Open: string, High: string, Low: string, Close: string, Volume: string, Adj Close: string]

## Bonus Question!
#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. [Check this link for a hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast)

If you get stuck on this, don't worry, just view the solutions.

root
 |-- summary: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



+-------+--------+--------+--------+--------+--------+
|summary|    Open|    High|     Low|   Close|  Volume|
+-------+--------+--------+--------+--------+--------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|    1258|
|   mean|   72.36|   72.84|   71.92|   72.39| 8222093|
| stddev|    6.77|    6.77|    6.74|    6.76| 4519781|
|    min|   56.39|   57.06|   56.30|   56.42| 2094900|
|    max|   90.80|   90.97|   89.25|   90.47|80898100|
+-------+--------+--------+--------+--------+--------+



#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [79]:
from pyspark.sql.functions import col

df1 = df.select((col("High") / col("Volume")).alias('HV Ratio'))
df1.show(10)

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714682786927E-6|
|6.290848662516662E-6|
| 4.66941298944916E-6|
| 7.36733843444859E-6|
|8.915604814435727E-6|
|8.644477449144044E-6|
|9.351828386844425E-6|
| 8.29141562102703E-6|
|7.712212051589609E-6|
|7.071764777688419...|
+--------------------+
only showing top 10 rows

#### What day had the Peak High in Price?

In [80]:
df.select('Date').orderBy('High', ascending=False).show(1)


+----------+
|      Date|
+----------+
|2015-01-13|
+----------+
only showing top 1 row

#### What is the mean of the Close column?

In [81]:
df.groupby().mean("Close").show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844997363553|
+-----------------+

#### What is the max and min of the Volume column?

In [82]:
from pyspark.sql.functions import min, max

In [83]:
df.select(max(col('Volume')), min(col('Volume'))).show(1)

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+

#### How many days was the Close lower than 60 dollars?

In [84]:
df.filter(col('Close') < 60).count()

81

#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [85]:
float(df.filter(col('High') > 80).count()) / df.count()

0.09141494435612083

#### What is the Pearson correlation between High and Volume?
#### [Hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameStatFunctions.corr)

In [86]:
df.corr('High', 'Volume')


-0.3384326061799556

#### What is the max High per year?

In [87]:
from pyspark.sql.functions import year

df.groupby(year('Date').alias('Year')).max('High').show()

+----+---------+
|Year|max(High)|
+----+---------+
|2015|    90.97|
|2013|    81.37|
|2014|    88.09|
|2012|     77.6|
|2016|    75.19|
+----+---------+

#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [88]:
from pyspark.sql.functions import year

df.groupby(month(col('Date')).alias('Month')).avg('Close').orderBy(month(col('Date'))).show()

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|    1| 71.4480196131338|
|    2|71.30680438169499|
|    3|71.77794376266337|
|    4|72.97361900692894|
|    5|72.30971685445533|
|    6| 72.4953774506191|
|    7|74.43971944078106|
|    8| 73.0298185521906|
|    9|72.18411782208611|
|   10| 71.5785454489968|
|   11|72.11108927207418|
|   12|72.84792482628012|
+-----+-----------------+

# Great Job!